In this notebook, you will learn to add AI options to a basic Teams-ai bot.

A Teams-ai application object can have an `ApplicationOptions.ai` property with `AIOption` type. A bot application with `ApplicationOptions.ai` property can work as a bot with AI abilities. For example, it can give a artificial response to your input according to prompts.

`AIOptions` consists of `planner`, `prompt` and `history`. `planner` will generate plan according to given prompts and chat history. `prompt` determines what prompts to use. We will show you how `planner` and `prompt` work.

## Prompt
In this section, we will introduce how to generate a prompts folder as a prompts base, which can be loaded by the `AIOption`, from a raw string.

## Planner
In this section, we will introduce how does a `planner` generates a plan.

First, we import needed packages

In [ ]:
# import modules from src
import src.config as config
from src.bot import *
from teams import AIHistoryOptions, AzureOpenAIPlanner, AzureOpenAIPlannerOptions, ConversationHistory, ConversationState, TempState, UserState
from botbuilder.schema import ChannelAccount, ConversationAccount
from unittest.mock import MagicMock 

Second, we initiate a `planner` object with Azure OpenAI keys and values, determining prompt folder.

In [ ]:
# register a planner
config.AZURE_OPENAI_KEY = 'b9e28116eeff4b129db5155fd261a541' # Azure OpenAI API key
config.AZURE_OPENAI_MODEL_DEPLOYMENT_NAME = 'gpt-35-turbo-zihch' # Azure OpenAI model deployment name
config.AZURE_OPENAI_ENDPOINT = 'https://azure-openai-zihch.openai.azure.com/' # Azure OpenAI endpoint
planner = AzureOpenAIPlanner(
    AzureOpenAIPlannerOptions(
        config.AZURE_OPENAI_KEY,
        config.AZURE_OPENAI_MODEL_DEPLOYMENT_NAME,
        config.AZURE_OPENAI_ENDPOINT,
        prompt_folder="src/prompts",
    )
)

Then, we mock a `TurnContext` object and a `TurnState` object. They will be used when generating a plan.

In [ ]:

# mock the TurnContext object
context=MagicMock(spec=TurnContext)

# mock the TurnState object
temp=TempState(input='hi', history='', output='')
user=UserState(__key__='mock_key', channel=ChannelAccount())
conversation=ConversationState(
    __key__='mock_key',
    account=ConversationAccount(),
    channel=ChannelAccount(),
    history=ConversationHistory()
)
state=TurnState(
    temp=temp,
    user=user,
    conversation=conversation
)

Last, we call `planner._prompt_manager.render_prompt()` and `planner.generate_plan()` to see what prompt content passed to planner and what plan is generated by planner.

In [ ]:

# render prompt
prompt="chat"
prompt_templates = await planner._prompt_manager.render_prompt(context, state, prompt)
print(f"prompt_templates:\n{prompt_templates}")

# generate plan
plan = await planner.generate_plan(
    context, state, prompt, history_options=AIHistoryOptions(assistant_history_type="text")
)
print(f"plan:\n{plan}")